In [18]:
from sage.all import *

In [23]:
# Define symbols for the quadratic rational Bezier curve
# In Sage, we use var() to declare symbolic variables
P = matrix([[var(f"P_{p}_{n}") for n in ["x", "y"]] for p in [0, 1, 2]])
w = vector(var("w0, w1, w2")) # Use a vector for the weights
t = var("t")

# The quadratic Bernstein basis polynomials as a vector
B = vector([(1 - t) ** 2, 2 * t * (1 - t), t**2])

# The parametric curve
# Note the use of diagonal_matrix() and vector/matrix multiplication
C = (P.transpose() * diagonal_matrix(w) * B) / (B * w)


C = C.subs({w[0]: 1, w[2]: 1})

show(C)


((2*P_1_x*(t - 1)*t*w1 - P_0_x*(t - 1)^2 - P_2_x*t^2)/(2*(t - 1)*t*w1 - (t - 1)^2 - t^2), (2*P_1_y*(t - 1)*t*w1 - P_0_y*(t - 1)^2 - P_2_y*t^2)/(2*(t - 1)*t*w1 - (t - 1)^2 - t^2))

In [36]:
assume(w[1]>0)
assume(w[1]<1)

x = C[0]
y = C[1]

area = (integrate(x*derivative(y,t)-y*derivative(x,t),t,0,1)/2).full_simplify()

show(area)

-1/2*((P_0_y*P_1_x - P_0_x*P_1_y + P_1_y*P_2_x - P_1_x*P_2_y)*w1^4 + 2*(P_0_y*P_1_x - P_0_x*P_1_y - (P_0_y - P_1_y)*P_2_x + (P_0_x - P_1_x)*P_2_y)*sqrt(-w1^2 + 1)*w1*arctan(sqrt(-w1^2 + 1)/(w1 + 1)) - (P_0_y*P_1_x - P_0_x*P_1_y + (P_0_y + P_1_y)*P_2_x - (P_0_x + P_1_x)*P_2_y)*w1^2 + P_0_y*P_2_x - P_0_x*P_2_y)/(w1^4 - 2*w1^2 + 1)

In [37]:
u, v = var("u, v")

l = u + v
k = u - v

# Weird factor required to make area = k * ||P_2-P_0||^2
weirdy_factor = 2 * k / ((l**3 + l) * atan2(1, l) - l**2)

subs = {
        w[1]: l/sqrt(1+l**2),
        # Constrain P_1 to be on the line equidistant from P_0 and P_2, parameterized by k.
        P[1, 0]: (P[0, 0] + P[2, 0]) / 2 + weirdy_factor * (P[0, 1] - P[2, 1]),
        P[1, 1]: (P[0, 1] + P[2, 1]) / 2 + weirdy_factor * (P[2, 0] - P[0, 0]),
}

Csub = C.subs(subs)

complex_form = arctan(1 / (l + sqrt(l**2 + 1)))
simple_form = atan2(1, l) / 2
areaSub = area.subs(subs).full_simplify().subs({complex_form: simple_form}).simplify()

show(areaSub)

1/2*((2*P_0_x^2 + 2*P_0_y^2 - (P_0_y*arctan2(1, u + v) + 4*P_0_x)*P_2_x + 2*P_2_x^2 + (P_0_x*arctan2(1, u + v) - 4*P_0_y)*P_2_y + 2*P_2_y^2)*u^2 - (2*P_0_x^2 + 2*P_0_y^2 + (P_0_y*arctan2(1, u + v) - 4*P_0_x)*P_2_x + 2*P_2_x^2 - (P_0_x*arctan2(1, u + v) + 4*P_0_y)*P_2_y + 2*P_2_y^2)*v^2 - P_0_y*P_2_x*arctan2(1, u + v) + P_0_x*P_2_y*arctan2(1, u + v) + (P_0_y*P_2_x - P_0_x*P_2_y)*u + (P_0_y*P_2_x - P_0_x*P_2_y - 2*(P_0_y*P_2_x*arctan2(1, u + v) - P_0_x*P_2_y*arctan2(1, u + v))*u)*v - 4*((P_0_x^2 + P_0_y^2 - 2*P_0_x*P_2_x + P_2_x^2 - 2*P_0_y*P_2_y + P_2_y^2)*u^3 - (P_0_x^2 + P_0_y^2 - 2*P_0_x*P_2_x + P_2_x^2 - 2*P_0_y*P_2_y + P_2_y^2)*u*v^2 - (P_0_x^2 + P_0_y^2 - 2*P_0_x*P_2_x + P_2_x^2 - 2*P_0_y*P_2_y + P_2_y^2)*v^3 + (P_0_x^2 + P_0_y^2 - 2*P_0_x*P_2_x + P_2_x^2 - 2*P_0_y*P_2_y + P_2_y^2)*u + ((P_0_x^2 + P_0_y^2 - 2*P_0_x*P_2_x + P_2_x^2 - 2*P_0_y*P_2_y + P_2_y^2)*u^2 - P_0_x^2 - P_0_y^2 + 2*P_0_x*P_2_x - P_2_x^2 + 2*P_0_y*P_2_y - P_2_y^2)*v)*arctan(1/(u + v + sqrt(u^2 + 2*u*v + v^2 + 1))))/(u^2*arctan2(1, u + v) + v^2*arctan2(1, u + v) + (2*u*arctan2(1, u + v) - 1)*v - u + arctan2(1, u + v))